# Import important libraries

In [1]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


#BILSTM

In [ ]:

class SERDataset(Dataset):
    def __init__(self, data_path, scaler=None, train=True):
        with open(data_path, 'r') as f:
            self.data = json.load(f)

        self.ids = list(self.data.keys())
        self.features = [np.array(self.data[key]["features"]) for key in self.ids]
        self.max_len = max([f.shape[0] for f in self.features])
        if train:
            self.valence = [self.data[key]["valence"] for key in self.ids]
            self.activation = [self.data[key]["activation"] for key in self.ids]
        else:
            self.valence = None
            self.activation = None

        self.scaler = scaler if scaler else StandardScaler()
        self.features = [self.scaler.fit_transform(f) for f in self.features]

        self.features = [np.pad(f, ((0, self.max_len - f.shape[0]), (0, 0)), mode='constant') for f in self.features]

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        if self.valence is not None:
            y_val = torch.tensor(self.valence[idx], dtype=torch.float32)
            y_act = torch.tensor(self.activation[idx], dtype=torch.float32)
            return x, y_val, y_act
        return x


class EmotionLSTM(nn.Module):
    def __init__(self, input_dim=26, hidden_dim=64, num_layers=2, dropout=0.3):
        super(EmotionLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.batch_norm = nn.BatchNorm1d(hidden_dim * 2)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, 2)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)

        h_n = h_n[-2:].permute(1, 0, 2).contiguous()
        h_n = h_n.view(x.size(0), -1)

        h_n = self.batch_norm(h_n)
        h_n = self.dropout(h_n)
        out = self.fc(h_n)
        return out



def train_model(train_loader, dev_loader, epochs=20, lr=0.001, device='cuda'):
    model = EmotionLSTM().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for x, y_val, y_act in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            x, y_val, y_act = x.to(device), y_val.to(device), y_act.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs[:, 0], y_val) + criterion(outputs[:, 1], y_act)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Evaluate on dev set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y_val, y_act in dev_loader:
                x, y_val, y_act = x.to(device), y_val.to(device), y_act.to(device)
                outputs = torch.sigmoid(model(x))
                preds = (outputs > 0.5).float()
                correct += ((preds[:, 0] == y_val) & (preds[:, 1] == y_act)).sum().item()
                total += y_val.size(0)

        accuracy = correct / total
        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Dev Accuracy: {accuracy:.4f}")

    return model


def generate_predictions(model, test_loader, test_path, device='cuda'):
    model.eval()
    results = {}

    with torch.no_grad():
        for idx, x in enumerate(test_loader):
            x = x.to(device)
            outputs = torch.sigmoid(model(x))
            preds = (outputs > 0.5).int()
            results[str(idx)] = {"valence": preds[0, 0].item(), "activation": preds[0, 1].item()}

    with open("submission.json", "w") as f:
        json.dump(results, f, indent=4)
    print("Predictions saved as submission.json")


train_dataset = SERDataset("train.json", train=True)
dev_dataset = SERDataset("dev.json", scaler=train_dataset.scaler, train=True)
test_dataset = SERDataset("ser_test_1.json", scaler=train_dataset.scaler, train=False)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = train_model(train_loader, dev_loader, epochs=20, lr=0.001, device=device)

generate_predictions(model, test_loader, "ser_test_1.json", device=device)


Epoch 1/20: 100%|██████████| 244/244 [00:08<00:00, 27.20it/s]


Epoch 1, Train Loss: 1.2756, Dev Accuracy: 0.4324


Epoch 2/20: 100%|██████████| 244/244 [00:06<00:00, 36.91it/s]


Epoch 2, Train Loss: 1.2290, Dev Accuracy: 0.4291


Epoch 3/20: 100%|██████████| 244/244 [00:06<00:00, 35.43it/s]


...

Epoch 18, Train Loss: 1.0008, Dev Accuracy: 0.4518


Epoch 19/20: 100%|██████████| 244/244 [00:07<00:00, 32.78it/s]


Epoch 19, Train Loss: 0.9715, Dev Accuracy: 0.4638


Epoch 20/20: 100%|██████████| 244/244 [00:07<00:00, 33.63it/s]


Epoch 20, Train Loss: 0.9426, Dev Accuracy: 0.5156
Predictions saved as submission.json


#CNN+BILSTM

In [ ]:
class SERDataset(Dataset):
    def __init__(self, data_path, scaler=None, train=True):
        with open(data_path, 'r') as f:
            self.data = json.load(f)

        self.ids = list(self.data.keys())
        self.features = [np.array(self.data[key]["features"]) for key in self.ids]
        self.max_len = max([f.shape[0] for f in self.features])

        if train:
            self.valence = [self.data[key]["valence"] for key in self.ids]
            self.activation = [self.data[key]["activation"] for key in self.ids]
        else:
            self.valence = None
            self.activation = None

        self.scaler = scaler if scaler else StandardScaler()
        self.features = [self.scaler.fit_transform(f) for f in self.features]

        self.features = [np.pad(f, ((0, self.max_len - f.shape[0]), (0, 0)), mode='constant') for f in self.features]

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        if self.valence is not None:
            y_val = torch.tensor(self.valence[idx], dtype=torch.float32)
            y_act = torch.tensor(self.activation[idx], dtype=torch.float32)
            return x, y_val, y_act
        return x


class CNNBiLSTM(nn.Module):
    def __init__(self, input_dim=26, hidden_dim=128, num_layers=2, dropout=0.3):
        super(CNNBiLSTM, self).__init__()

        # CNN Layer: Local feature extraction
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.batch_norm = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()

        # BiLSTM Layer: Temporal modeling
        self.lstm = nn.LSTM(128, hidden_dim, num_layers, batch_first=True, bidirectional=True)

        # Fully Connected Layer: Output (valence & activation)
        self.fc = nn.Linear(hidden_dim * 2, 2)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # CNN layers
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.batch_norm(x)
        x = x.permute(0, 2, 1)

        # BiLSTM
        _, (h_n, _) = self.lstm(x)

        # Get last forward & backward hidden states
        h_n = h_n[-2:].permute(1, 0, 2).contiguous()
        h_n = h_n.view(x.size(0), -1)

        # Dropout and output
        h_n = self.dropout(h_n)
        out = self.fc(h_n)
        return out


def train_model(train_loader, dev_loader, epochs=30, lr=0.0005, device='cuda'):
    model = CNNBiLSTM().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for x, y_val, y_act in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            x, y_val, y_act = x.to(device), y_val.to(device), y_act.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs[:, 0], y_val) + criterion(outputs[:, 1], y_act)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Evaluate on dev set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y_val, y_act in dev_loader:
                x, y_val, y_act = x.to(device), y_val.to(device), y_act.to(device)
                outputs = torch.sigmoid(model(x))
                preds = (outputs > 0.5).float()
                correct += ((preds[:, 0] == y_val) & (preds[:, 1] == y_act)).sum().item()
                total += y_val.size(0)

        accuracy = correct / total
        scheduler.step(accuracy)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Dev Accuracy: {accuracy:.4f}")

    return model


def generate_predictions(model, test_loader, output_file, device='cuda'):
    model.eval()
    results = {}

    with torch.no_grad():
        for idx, x in enumerate(test_loader):
            x = x.to(device)
            outputs = torch.sigmoid(model(x))
            preds = (outputs > 0.5).int()
            results[str(idx)] = {"valence": preds[0, 0].item(), "activation": preds[0, 1].item()}

    with open(output_file, "w") as f:
        json.dump(results, f, indent=4)
    print(f"Predictions saved as {output_file}")


train_dataset = SERDataset("train.json", train=True)
dev_dataset = SERDataset("dev.json", scaler=train_dataset.scaler, train=True)
test_dataset = SERDataset("ser_test_1.json", scaler=train_dataset.scaler, train=False)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = train_model(train_loader, dev_loader, epochs=30, lr=0.0005, device=device)

generate_predictions(model, test_loader, "submission.json", device=device)


Epoch 1/30: 100%|██████████| 244/244 [00:25<00:00,  9.41it/s]


Epoch 1, Train Loss: 1.2445, Dev Accuracy: 0.4545


Epoch 2/30: 100%|██████████| 244/244 [00:24<00:00,  9.84it/s]


Epoch 2, Train Loss: 1.1982, Dev Accuracy: 0.4683


Epoch 3/30: 100%|██████████| 244/244 [00:25<00:00,  9.66it/s]


...

Epoch 28, Train Loss: 0.1188, Dev Accuracy: 0.5102


Epoch 29/30: 100%|██████████| 244/244 [00:27<00:00,  8.80it/s]


Epoch 29, Train Loss: 0.0685, Dev Accuracy: 0.5218


Epoch 30/30: 100%|██████████| 244/244 [00:27<00:00,  8.80it/s]


Epoch 30, Train Loss: 0.0513, Dev Accuracy: 0.5215
Predictions saved as submission.json


#Distilbert

In [ ]:
!pip install transformers datasets torch accelerate scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0


In [ ]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizerFast
from tqdm import tqdm


class SERDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=128, train=True):
        with open(data_path, 'r') as f:
            self.data = json.load(f)

        self.ids = list(self.data.keys())
        self.features = [" ".join(map(str, self.data[key]["features"])) for key in self.ids]

        if train:
            self.valence = [self.data[key]["valence"] for key in self.ids]
            self.activation = [self.data[key]["activation"] for key in self.ids]
        else:
            self.valence = None
            self.activation = None

        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.features[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        if self.valence is not None:
            y_val = torch.tensor(self.valence[idx], dtype=torch.float32)
            y_act = torch.tensor(self.activation[idx], dtype=torch.float32)
            return input_ids, attention_mask, y_val, y_act
        return input_ids, attention_mask


class TransformerSER(nn.Module):
    def __init__(self, hidden_dim=256, dropout=0.3):
        super(TransformerSER, self).__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.bert.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        return self.fc(cls_output)


def train_model(train_loader, dev_loader, epochs=10, lr=5e-5, device='cuda'):
    model = TransformerSER().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.7)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for input_ids, attention_mask, y_val, y_act in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            y_val, y_act = y_val.to(device), y_act.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs[:, 0], y_val) + criterion(outputs[:, 1], y_act)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            for input_ids, attention_mask, y_val, y_act in dev_loader:
                input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
                y_val, y_act = y_val.to(device), y_act.to(device)
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs[:, 0], y_val) + criterion(outputs[:, 1], y_act)
                val_loss += loss.item()

        scheduler.step()
        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(dev_loader):.4f}")

    return model


def generate_predictions(model, test_loader, output_file, device='cuda'):
    model.eval()
    results = {}

    with torch.no_grad():
        for idx, (input_ids, attention_mask) in enumerate(test_loader):
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            outputs = model(input_ids, attention_mask)
            preds = outputs.cpu().detach().numpy().tolist()
            results[str(idx)] = {"valence": preds[0][0], "activation": preds[0][1]}

    with open(output_file, "w") as f:
        json.dump(results, f, indent=4)
    print(f"Predictions saved as {output_file}")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_dataset = SERDataset("/content/train.json", tokenizer, train=True)
dev_dataset = SERDataset("/content/dev.json", tokenizer, train=True)
test_dataset = SERDataset("/content/ser_test_1.json", tokenizer, train=False)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model = train_model(train_loader, dev_loader, epochs=10, lr=5e-5, device=device)

generate_predictions(model, test_loader, "submission.json", device=device)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Epoch 1/10:  86%|████████▋ | 421/488 [1:08:12<10:44,  9.62s/it]

#Transformer SER Paper method

In [3]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

def load_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

train_data = load_json('train.json')
dev_data = load_json('dev.json')

def process_data(data, is_test=False):
    X, y = [], []
    for key, value in data.items():
        X.append(torch.tensor(value["features"], dtype=torch.float32))
        if not is_test and "valence" in value and "activation" in value:
            y.append(torch.tensor([value["valence"], value["activation"]], dtype=torch.float32))
    X_padded = pad_sequence(X, batch_first=True)
    return (X_padded, torch.stack(y) if y else None)

X_train, y_train = process_data(train_data)
X_dev, y_dev = process_data(dev_data)

class SERDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = X
        self.y = y if y is not None else [torch.tensor([0.0, 0.0], dtype=torch.float32)] * len(X)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = SERDataset(X_train, y_train)
dev_dataset = SERDataset(X_dev, y_dev)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False)

class TransformerSER(nn.Module):
    def __init__(self, input_dim=26, d_model=128, num_heads=4, num_layers=2, ff_dim=256):
        super(TransformerSER, self).__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=ff_dim, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(d_model, 2)

    def forward(self, x):
        x = self.input_proj(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        return self.fc(x)

def train_model(model, train_loader, dev_loader, epochs=20, lr=0.001, device="cpu"):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    best_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        dev_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in dev_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                output = model(X_batch)
                loss = criterion(output, y_batch)
                dev_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Dev Loss: {dev_loss/len(dev_loader):.4f}")
        if dev_loss < best_loss:
            best_loss = dev_loss
            torch.save(model.state_dict(), "best_model.pth")
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerSER().to(device)
model = train_model(model, train_loader, dev_loader, epochs=20, lr=0.0005, device=device)

test_data = load_json('/content/ser_test_2.json')
X_test, _ = process_data(test_data, is_test=True)
test_dataset = SERDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.load_state_dict(torch.load("best_model.pth"))
model.eval()

predictions = {}
with torch.no_grad():
    for idx, (X_batch, _) in enumerate(test_loader):
        X_batch = X_batch.to(device)
        output = model(X_batch)
        preds = output.cpu().tolist()
        for i, pred in enumerate(preds):
            predictions[str(idx * 32 + i)] = {"valence": pred[0], "activation": pred[1]}

with open("submission_ser_2_json.json", "w") as f:
    json.dump(predictions, f)


Epoch 1/20, Train Loss: 0.2205, Dev Loss: 0.2014
Epoch 2/20, Train Loss: 0.2043, Dev Loss: 0.1889
Epoch 3/20, Train Loss: 0.2020, Dev Loss: 0.1916
Epoch 4/20, Train Loss: 0.1960, Dev Loss: 0.1946
Epoch 5/20, Train Loss: 0.1940, Dev Loss: 0.1816
Epoch 6/20, Train Loss: 0.1903, Dev Loss: 0.1883
Epoch 7/20, Train Loss: 0.1891, Dev Loss: 0.2021
Epoch 8/20, Train Loss: 0.1900, Dev Loss: 0.1805
Epoch 9/20, Train Loss: 0.1855, Dev Loss: 0.1808
Epoch 10/20, Train Loss: 0.1862, Dev Loss: 0.1913
Epoch 11/20, Train Loss: 0.1838, Dev Loss: 0.1756
Epoch 12/20, Train Loss: 0.1811, Dev Loss: 0.1839
Epoch 13/20, Train Loss: 0.1782, Dev Loss: 0.1752
Epoch 14/20, Train Loss: 0.1784, Dev Loss: 0.1803
Epoch 15/20, Train Loss: 0.1764, Dev Loss: 0.1813
Epoch 16/20, Train Loss: 0.1762, Dev Loss: 0.1779
Epoch 17/20, Train Loss: 0.1748, Dev Loss: 0.1848
Epoch 18/20, Train Loss: 0.1753, Dev Loss: 0.1790
Epoch 19/20, Train Loss: 0.1719, Dev Loss: 0.1820
Epoch 20/20, Train Loss: 0.1720, Dev Loss: 0.1809


<ipython-input-3-8e6712b23e66>:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


In [4]:
test_data = load_json('/content/ser_test_2.json')
X_test, _ = process_data(test_data, is_test=True)
test_dataset = SERDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.load_state_dict(torch.load("best_model.pth"))
model.eval()

predictions = {}
with torch.no_grad():
    for idx, (X_batch, _) in enumerate(test_loader):
        X_batch = X_batch.to(device)
        output = model(X_batch)
        preds = output.cpu().tolist()
        for i, pred in enumerate(preds):
            predictions[str(idx * 32 + i)] = {"valence": pred[0], "activation": pred[1]}

with open("submission_ser_2_json.json", "w") as f:
    json.dump(predictions, f)

<ipython-input-4-99f64759a841>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


# Incase submissions are not rounded values


In [6]:
import json

with open('/content/submission_ser_2_json.json', 'r') as file:
    submissions = json.load(file)

def round_values(submissions):
    for key, value in submissions.items():
        value['valence'] = 1 if value['valence'] >= 0.5 else 0
        value['activation'] = 1 if value['activation'] >= 0.5 else 0
    return submissions

rounded_submissions = round_values(submissions)

with open('final_submissions_rounded_ser_2.json', 'w') as file:
    json.dump(rounded_submissions, file, indent=2)

print(json.dumps(rounded_submissions, indent=2))


{
  "0": {
    "valence": 1,
    "activation": 0
  },
  "1": {
    "valence": 0,
    "activation": 0
  },
  "2": {
    "valence": 0,
    "activation": 0
  },
  "3": {
    "valence": 1,
    "activation": 0
  },
  "4": {
    "valence": 0,
    "activation": 0
  },
  "5": {
    "valence": 1,
    "activation": 1
  },
  "6": {
    "valence": 1,
    "activation": 1
  },
  "7": {
    "valence": 1,
    "activation": 1
  },
  "8": {
    "valence": 1,
    "activation": 0
  },
  "9": {
    "valence": 1,
    "activation": 1
  },
  "10": {
    "valence": 0,
    "activation": 0
  },
  "11": {
    "valence": 1,
    "activation": 0
  },
  "12": {
    "valence": 0,
    "activation": 0
  },
  "13": {
    "valence": 0,
    "activation": 0
  },
  "14": {
    "valence": 1,
    "activation": 1
  },
  "15": {
    "valence": 1,
    "activation": 0
  },
  "16": {
    "valence": 0,
    "activation": 1
  },
  "17": {
    "valence": 1,
    "activation": 1
  },
  "18": {
    "valence": 1,
    "activation": 1
  },
 